# 把数据转存到tfrecord，然后从tfrecord拿数据训练网络


具体一些细节和疑问：
所谓那套mnist的jpg从哪取:
如果自己转，需要面临的问题有，txt文件和里边对应标签的处理。繁琐一些。
先找到他提供的数据

单次拿数据能否超过capacity数量？测试使用coord是否多此一举？还是coord反而是为了解决这个问题而存在的？

可以再多练的：
tfrecord基本操作。
管道和线程


In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

In [2]:
INPUT_SIZE = 784
OUTPUT_SIZE = 10
L1_SIZE = 500

BATCH_SIZE = 200
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.99
REGULAR_SCALE = 0.0001
STEPS = 50000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = './model/'
MODEL_NAME = 'mnist_model'

DATA_DIR = 'MNIST_data'#本例没太大用，走tfrecord了

train_num_examples = 60000#也就是mnist.train.num_examples返回的那个数值。



image_train_path = 'tensorflow_mooc/fc4/mnist_data_jpg/mnist_train_jpg_60000/'
label_train_path = 'tensorflow_mooc/fc4/mnist_data_jpg/mnist_train_jpg_60000.txt'
image_test_path = 'tensorflow_mooc/fc4/mnist_data_jpg/mnist_test_jpg_10000/'
label_test_path = 'tensorflow_mooc/fc4/mnist_data_jpg/mnist_test_jpg_10000.txt'

tfRecord_train = 'tensorflow_mooc/fc4/data/mnist_train.tfrecords'
tfRecord_test = 'tensorflow_mooc/fc4/data/mnist_test.tfrecords'

data_path = 'tensorflow_mooc/fc4/my_mnist_tfRecord/'#同tensorflow_mooc/fc4/data
resize_height = 28
resize_width = 28


In [3]:
def read_tfRecord(tfRecord_path):
    filename_queue = tf.train.string_input_producer([tfRecord_path])#这应该是拿着路径生成一个文件名的队列
    reader = tf.TFRecordReader()#reader对象
    _, serialized_example = reader.read(filename_queue)#读出来，已经是example了？
    features = tf.parse_single_example(serialized_example, features={'label':tf.FixedLenFeature([10],tf.int64),
                                                                    'img_raw':tf.FixedLenFeature([],tf.string)})
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img.set_shape([784])
    img = tf.cast(img, tf.float32) * (1./255.)
    label = tf.cast(features['label'], tf.float32)
    return img, label
    
def get_tfRecord(num, isTrain = True):
    if isTrain:
        tfRecord_path = tfRecord_train
    else:
        tfRecord_path = tfRecord_test
    img, label = read_tfRecord(tfRecord_path)
    
    #他说”顺序取出capacity个，打乱顺序“
    #取出capacity个打乱？从总样本中随机取capacity个？前者没意义，打乱再取，是随机batch，取再打乱，有什么意义？
    #测试，如果是他说的，很容易验证，比如第二批，内部怎么打乱，所有元素都是按顺序取的第二批。把size弄成1好观察。
    
    #batch_size = num是穿进来的参数，和capacity不同，capacity更像是池内部的属性，这个应该是每次取出的
    #举例，batch_size=100,那么这个容器就应该是1000个，900个，800个，700个，达到min，然后填满？
    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                    batch_size = num,
                                                   num_threads = 2,
                                                   capacity = 1000,
                                                   min_after_dequeue = 700)#因为是动态的，好下功能是池子剩余不到700就再填充。
    
    return img_batch, label_batch
    
    
def write_tfRecord(tfRecordName, image_path, label_path):
    writer = tf.python_io.TFRecordWriter(tfRecordName)
    num_pic = 0
    f = open(label_path, 'r')
    contents = f.readlines()
    f.close()
    for content in contents:#遍历所有行（数据和标签）
        value = content.split()#空格分割
        img_path = image_path + value[0]
        img = Image.open(img_path)
        img_raw = img.tobytes()
        labels = [0] * 10#one-hot形式
        labels[int(value[1])] = 1
        example = tf.train.Example(features = tf.train.Features(feature={#图片+标签
            'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw])),
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value=labels))
        }))
        writer.write(example.SerializeToString())
        num_pic += 1
        print('the number of picture is ',num_pic)
    writer.close()
    print('write tfrecord successfully')
        
def generate_tfRecord():
    isExists = os.path.exists(data_path)
    if not isExists:
        os.makedirs(data_path)
    else:
        print('dir exists')
    #从后两者取数据生成前者，所以，我还是得先准备好数据放到指定路径，目前没有
    write_tfRecord(tfRecord_train, image_train_path, label_train_path)
    write_tfRecord(tfRecord_test, image_test_path, label_test_path)


In [4]:
def get_weight(shape, regular_scale):
    w = tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    if regular_scale != None:
        tf.add_to_collection('regular_loss',tf.contrib.layers.l2_regularizer(regular_scale)(w))
    return w
def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b
def net(x,regular_scale):
    w1 = get_weight([INPUT_SIZE,L1_SIZE], regular_scale)
    b1 = get_bias([L1_SIZE])
    l1 = tf.nn.relu(tf.matmul(x,w1)+b1)
    
    w2 = get_weight([L1_SIZE, OUTPUT_SIZE], regular_scale)
    b2 = get_bias([OUTPUT_SIZE])
    y = tf.matmul(l1,w2)+b2
    return y  

In [5]:
def train():
    x = tf.placeholder(tf.float32, [None,INPUT_SIZE])
    y_ = tf.placeholder(tf.float32, [None,OUTPUT_SIZE])
    y = net(x,REGULAR_SCALE)
    global_step = tf.Variable(0, trainable = False)
    
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y, labels = tf.argmax(y_,1))#label需要转换成非one-hot
    cem = tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('regular_loss'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,
                                               train_num_examples / BATCH_SIZE,
                                              LEARNING_RATE_DECAY,
                                               staircase = True)
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_op, ema_op]):
        train_op = tf.no_op(name='train')#train_op重名，也可以改名
    
    saver = tf.train.Saver()
    img_batch, label_batch = get_tfRecord(BATCH_SIZE, isTrain = True)#替换原来的数据来源
    
    
    writer = tf.summary.FileWriter('model_mnist_in_tfrecord_graph')
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        #断点续训：
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            
        #如何使for循环中的img_batch每次不同？用sess.run()从管道取的,赋值给xs，ys
        #临时开启多线程，提升管道的读取效率。
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess = sess, coord = coord)
        
        for i in range(STEPS):
            xs, ys = sess.run([img_batch, label_batch])#data.train.next_batch(BATCH_SIZE)
            _, loss_val, step = sess.run([train_op,loss,global_step],feed_dict={x:xs, y_:ys})#替换原来的数据来源
            if i % 1000 == 0:
                print('after %d steps, total loss is %g'%(step,loss_val))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)
        coord.request_stop()
        coord.join(threads)

In [6]:
def main():
    train()
if __name__ == '__main__':
    main()

INFO:tensorflow:Restoring parameters from ./model/mnist_model-54002
after 54003 steps, total loss is 0.124507
after 55003 steps, total loss is 0.122255
after 56003 steps, total loss is 0.118437
after 57003 steps, total loss is 0.128632
after 58003 steps, total loss is 0.120297
after 59003 steps, total loss is 0.121506
after 60003 steps, total loss is 0.129775
after 61003 steps, total loss is 0.122284
after 62003 steps, total loss is 0.121707
after 63003 steps, total loss is 0.117267
after 64003 steps, total loss is 0.120999
after 65003 steps, total loss is 0.116291
after 66003 steps, total loss is 0.124315
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


KeyboardInterrupt: 

In [8]:
#测试

import tensorflow as tf
import time
TEST_INTERVAL_SECS = 5
TEST_NUM = 10000
def test():
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, INPUT_SIZE])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_SIZE])
        y = net(x,None)#Net结构是直接读取的定义，ema的变量都是从这找的
        
        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        #这是一套映射：如果变量有ema，就读ema，否则，用普通。
        #用ema resotre来初始化saver，影响saver.restore的读取内容吗？也就是说，能读取bias吗？
        #根据形状判断（因为变量名不可读），应该是有bias的
        ema_restore = ema.variables_to_restore()
        #打印的含义？为何有Exponential，是已经读到了？还是按规则自己假设的变量名？应该是按规则预设的变量名
        print('ema_restore length:',len(ema_restore))
        print('ema_restore:',ema_restore)
        saver = tf.train.Saver(ema_restore)
        print('saver:',saver)#<class 'tensorflow.python.training.saver.Saver'>
        print('saver print end')
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        
        img_batch, label_batch = get_tfRecord(TEST_NUM, isTrain = False)#取数据流程同train
        #为了测这个，可以写个for循环，任意次
#         while True:#把N秒一次改为总共一次
        for i in range(10):
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)#他只是从proto拿了路径，还是自己的restore
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    
                    #在这个大循环下，使用这个模式是否多此一举？
                    #capacity只有1000的，你这要获取10000？拿得出来？
                    #如果不用，会怎么样？
                    
                    coord = tf.train.Coordinator()
                    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
                    xs, ys = sess.run([img_batch, label_batch])
                    accuracy_score = sess.run(accuracy,feed_dict={x:xs,y_:ys})
                    
                    print('after % steps, accuracy is %f'%(global_step,accuracy_score))

                    coord.request_stop()
                    coord.join(threads)
                    
                else:
                    print('no ckpt')
                    return
            time.sleep(TEST_INTERVAL_SECS)
def main():
    test()
if __name__ == '__main__':
    main()

ema_restore length: 4
ema_restore: {'Variable_1/ExponentialMovingAverage': <tf.Variable 'Variable_1:0' shape=(500,) dtype=float32_ref>, 'Variable_3/ExponentialMovingAverage': <tf.Variable 'Variable_3:0' shape=(10,) dtype=float32_ref>, 'Variable/ExponentialMovingAverage': <tf.Variable 'Variable:0' shape=(784, 500) dtype=float32_ref>, 'Variable_2/ExponentialMovingAverage': <tf.Variable 'Variable_2:0' shape=(500, 10) dtype=float32_ref>}
saver: <tensorflow.python.training.saver.Saver object at 0x7f4b0c9f0780>
saver print end
INFO:tensorflow:Restoring parameters from ./model/mnist_model-66003
after 66003teps, accuracy is 0.981100
INFO:tensorflow:Restoring parameters from ./model/mnist_model-66003
after 66003teps, accuracy is 0.981200
INFO:tensorflow:Restoring parameters from ./model/mnist_model-66003
after 66003teps, accuracy is 0.981500
INFO:tensorflow:Restoring parameters from ./model/mnist_model-66003
after 66003teps, accuracy is 0.981500
INFO:tensorflow:Restoring parameters from ./model

# 做一套小数据集的tfrecord，做测试

In [4]:
mini_data_path = 'tensorflow_mooc/fc4/mini_tfRecord/'#同tensorflow_mooc/fc4/data
mini_tfRecord_train = 'tensorflow_mooc/fc4/mini_tfRecord/mini_mnist_train.tfrecords'
mini_tfRecord_test = 'tensorflow_mooc/fc4/mini_tfRecord/mini_mnist_test.tfrecords'

MINI_BATCH_NUM = 20

def write_tfRecord_mini(tfRecordName, image_path, label_path):
    writer = tf.python_io.TFRecordWriter(tfRecordName)
    num_pic = 0
    f = open(label_path, 'r')
#     contents = f.readlines()
    contents = [f.readline() for x in range(MINI_BATCH_NUM)]
    f.close()
    for content in contents[:MINI_BATCH_NUM]:#mini，做少点
        value = content.split()#空格分割
        img_path = image_path + value[0]
        img = Image.open(img_path)
        img_raw = img.tobytes()
        labels = [0] * 10#one-hot形式
        labels[int(value[1])] = 1
        example = tf.train.Example(features = tf.train.Features(feature={#图片+标签
            'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw])),
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value=labels))
        }))
        writer.write(example.SerializeToString())
        num_pic += 1
        print('the number of picture is ',num_pic)
    writer.close()
    print('write tfrecord successfully')
        
def generate_tfRecord_mini():
    isExists = os.path.exists(mini_data_path)
    if not isExists:
        os.makedirs(mini_data_path)
    else:
        print('dir exists')
    write_tfRecord_mini(mini_tfRecord_train, image_train_path, label_train_path)
    write_tfRecord_mini(mini_tfRecord_test, image_test_path, label_test_path)
generate_tfRecord_mini()

dir exists
the number of picture is  1
the number of picture is  2
the number of picture is  3
the number of picture is  4
the number of picture is  5
the number of picture is  6
the number of picture is  7
the number of picture is  8
the number of picture is  9
the number of picture is  10
the number of picture is  11
the number of picture is  12
the number of picture is  13
the number of picture is  14
the number of picture is  15
the number of picture is  16
the number of picture is  17
the number of picture is  18
the number of picture is  19
the number of picture is  20
write tfrecord successfully
the number of picture is  1
the number of picture is  2
the number of picture is  3
the number of picture is  4
the number of picture is  5
the number of picture is  6
the number of picture is  7
the number of picture is  8
the number of picture is  9
the number of picture is  10
the number of picture is  11
the number of picture is  12
the number of picture is  13
the number of picture 

In [ ]:
#label应该是所有样本的label都算，raw也是。运行卡死了。可能train六万个数据太大了，尝试做一个小文件
#20个数据仍然会卡？
#不能直接run提取？
def read_tfRecord_mini(tfRecord_path):
    filename_queue = tf.train.string_input_producer([tfRecord_path])#这应该是拿着路径生成一个文件名的队列
    reader = tf.TFRecordReader()#reader对象
    _, serialized_example = reader.read(filename_queue)#读出来，已经是example了？
    features = tf.parse_single_example(serialized_example, features={'label':tf.FixedLenFeature([10],tf.int64),
                                                                    'img_raw':tf.FixedLenFeature([],tf.string)})
    print(type(features))
    print(features)
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    print('img:',img)
    img.set_shape([784])
    print('reshape img:',img)
    img = tf.cast(img, tf.float32) * (1./255.)
    print('cast img:',img)
    label = tf.cast(features['label'], tf.float32)
    return img, label

img,label = read_tfRecord_mini(mini_tfRecord_train)
print(img)
print(img.shape)
print(label)

with tf.Session() as sess:
    print('run img:',sess.run(label))
#     print('run img:',sess.run(img))

<class 'dict'>
{'img_raw': <tf.Tensor 'ParseSingleExample/Squeeze_img_raw:0' shape=() dtype=string>, 'label': <tf.Tensor 'ParseSingleExample/Squeeze_label:0' shape=(10,) dtype=int64>}
img: Tensor("DecodeRaw:0", shape=(?,), dtype=uint8)
reshape img: Tensor("DecodeRaw:0", shape=(784,), dtype=uint8)
cast img: Tensor("mul:0", shape=(784,), dtype=float32)
Tensor("mul:0", shape=(784,), dtype=float32)
(784,)
Tensor("Cast_1:0", shape=(10,), dtype=float32)


In [ ]:
#运行卡死了。可能train六万个数据太大了，尝试做一个小文件
    
#他说”顺序取出capacity个，打乱顺序“
#取出capacity个打乱？从总样本中随机取capacity个？前者没意义，打乱再取，是随机batch，取再打乱，有什么意义？
#测试，如果是他说的，很容易验证，比如第二批，内部怎么打乱，所有元素都是按顺序取的第二批。把size弄成1好观察。

#batch_size = num是穿进来的参数，和capacity不同，capacity更像是池内部的属性，这个应该是每次取出的
#举例，batch_size=100,那么这个容器就应该是1000个，900个，800个，700个，达到min，然后填满？
def get_tfRecord_mini(num, isTrain = True):
    if isTrain:
        tfRecord_path = tfRecord_train
    else:
        tfRecord_path = tfRecord_test
    img, label = read_tfRecord_mini(tfRecord_path)

    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                    batch_size = num,
                                                   num_threads = 2,
                                                   capacity = 2,
                                                   min_after_dequeue = 1)
    return img_batch, label_batch

#测试，一次取3个，大于capacity的2个，会怎么样
img_batch, label_batch = get_tfRecord_mini(2,True)

with tf.Session() as sess:
    print(sess.run(img_batch))
    print(sess.run(label_batch))

# 基本接口和示例

In [3]:
writer = tf.python_io.TFRecordWriter('recordname')

In [3]:
#测coord
#普通python循环在这里不行，既然他也绑定sess，肯定还是得用tf的操作
#不好直观测，这个需要多个线程，本例是用的start_queue_runners直接黑盒做的。
a = tf.Variable(1)
op = tf.assign_add(a,1)
b = tf.Variable(1)
op2 = tf.assign_add(b,1)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(a))
    sess.run(op)
    print(sess.run(a))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
#     for i in range(1000):
#         print(sess.run(a),end=',')
#         sess.run(op)
#     for i in range(1000):
#         print(sess.run(b),end=',')
#         sess.run(op2)
    coord.request_stop()
    coord.join(threads)

1
2


In [18]:
# help(tf.train.Coordinator)
#这个协调器打开以后，打开多个线程，能等待他们一起结束
# .|  #### Usage:
#  |  
#  |  ```python
#  |  # Create a coordinator.
#  |  coord = Coordinator()
#  |  # Start a number of threads, passing the coordinator to each of them.
#  |  ...start thread 1...(coord, ...)
#  |  ...start thread N...(coord, ...)
#  |  # Wait for all the threads to terminate.
#  |  coord.join(threads)
#  |  ```

In [17]:
help(tf.train.start_queue_runners)

Help on function start_queue_runners in module tensorflow.python.training.queue_runner_impl:

start_queue_runners(sess=None, coord=None, daemon=True, start=True, collection='queue_runners')
    Starts all queue runners collected in the graph.
    
    This is a companion method to `add_queue_runner()`.  It just starts
    threads for all queue runners collected in the graph.  It returns
    the list of all threads.
    
    Args:
      sess: `Session` used to run the queue ops.  Defaults to the
        default session.
      coord: Optional `Coordinator` for coordinating the started threads.
      daemon: Whether the threads should be marked as `daemons`, meaning
        they don't block program exit.
      start: Set to `False` to only create the threads, not start them.
      collection: A `GraphKey` specifying the graph collection to
        get the queue runners from.  Defaults to `GraphKeys.QUEUE_RUNNERS`.
    
    Raises:
      ValueError: if `sess` is None and there isn't any de